# 0.5 Generate Noisy Data and Fitting

This notebook is used to generate and fit all the data required for the paper. We will generate data for the following noise cases: 

1, 2, 3, 4, 5, 6, 7 STD

## Imports

In [1]:
import sys

sys.path.append("../../")
sys.path.append("/home/ferroelectric/m3_learning/m3_learning/src")

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from m3_learning.be.dataset import BE_Dataset
from m3_learning.viz.printing import printer
from m3_learning.be.nn import SHO_fit_func_nn, SHO_Model

2023-05-14 17:27:13.735198: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 17:27:13.852305: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-14 17:27:14.288833: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ferroelectric/anaconda3/envs/rapidfitting/lib/python3.9/site-packages/cv2/../../lib64:
2023-05-14 17:27:14.288881: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic libr

## Loads the Dataset

In [3]:
# Specify the filename and the path to save the file
filename = "data_raw_original.h5"
save_path = "./Data"

data_path = save_path + "/" + filename

# instantiate the dataset object
dataset = BE_Dataset(data_path, resample_bins=None, SHO_fit_func_NN=SHO_fit_func_nn)

# print the contents of the file
dataset.print_be_tree()

/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS
    ├ UDVS_Indices
Datasets and datagroups within the file:
------------------------------------
/
/Measurement_000
/Measurement_000/Channel_000
/Measurement_000/Channel_000/Bin_FFT
/Measurement_000/Chann

## Generates Noisy Data

This function will generate noisy records and save them as an h5_main file in the USID format. This allows the data to be computed with the Pycroscopy SHO Fitter. 

In [4]:
 noise_STD = np.std(dataset.get_original_data)

In [12]:
dataset.generate_noisy_data_records(noise_levels = np.arange(1,11), verbose=True, noise_STD=5e-4)

The STD of the data is: 0.0005
Adding noise level 1
Adding noise level 2
Adding noise level 3
Adding noise level 4
Adding noise level 5
Adding noise level 6
Adding noise level 7
Adding noise level 8
Adding noise level 9
Adding noise level 10


## SHO fits on all the datasets

This will take some time, Each fit takes about 10 minutes to complete. 

In [13]:
out = [f"Noisy_Data_{i}" for i in np.arange(1,11)]
out.append("Raw_Data")

for data in out:
    print(f"Fitting {data}")
    dataset.SHO_Fitter(dataset = data, h5_sho_targ_grp = f"{data}_SHO_Fit", max_mem=1024*64, max_cores= 20)

Fitting Noisy_Data_1
Working on:
./Data/data_raw_original.h5
['Y', 'X'] [60, 60]


SHO Fits will be written to:
./Data/data_raw_original.h5


Consider calling test() to check results before calling compute() which computes on the entire dataset and writes results to the HDF5 file
	This class (likely) supports interruption and resuming of computations!
	If you are operating in a python console, press Ctrl+C or Cmd+C to abort
	If you are in a Jupyter notebook, click on "Kernel">>"Interrupt"
	If you are operating on a cluster and your job gets killed, re-run the job to resume

Rank 0 finished parallel computation
Rank 0 - 3% complete. Time remaining: 2.34 mins
Rank 0 finished parallel computation
Rank 0 - 7% complete. Time remaining: 2.17 mins
Rank 0 finished parallel computation
Rank 0 - 11% complete. Time remaining: 2.03 mins
Rank 0 finished parallel computation
Rank 0 - 15% complete. Time remaining: 1.93 mins
Rank 0 finished parallel computation
Rank 0 - 19% complete. Time remaining: 1

/home/ferroelectric/anaconda3/envs/rapidfitting/lib/python3.9/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')
/home/ferroelectric/anaconda3/envs/rapidfitting/lib/python3.9/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')


Rank 0 - 3% complete. Time remaining: 5.33 mins
Rank 0 - 6% complete. Time remaining: 3.83 mins
Rank 0 - 10% complete. Time remaining: 3.41 mins
Rank 0 - 13% complete. Time remaining: 3.15 mins
Rank 0 - 16% complete. Time remaining: 2.88 mins
Rank 0 - 20% complete. Time remaining: 2.3 mins
Rank 0 - 23% complete. Time remaining: 2.18 mins
Rank 0 - 26% complete. Time remaining: 1.99 mins
Rank 0 - 30% complete. Time remaining: 1.84 mins
Rank 0 - 33% complete. Time remaining: 1.8 mins
Rank 0 - 36% complete. Time remaining: 1.79 mins
Rank 0 - 40% complete. Time remaining: 1.7 mins
Rank 0 - 43% complete. Time remaining: 1.61 mins
Rank 0 - 46% complete. Time remaining: 1.51 mins
Rank 0 - 50% complete. Time remaining: 1.38 mins
Rank 0 - 53% complete. Time remaining: 1.23 mins
Rank 0 - 56% complete. Time remaining: 1.17 mins
Rank 0 - 60% complete. Time remaining: 1.1 mins
Rank 0 - 63% complete. Time remaining: 1.05 mins
Rank 0 - 66% complete. Time remaining: 58.48 sec
Rank 0 - 70% complete. Tim

/home/ferroelectric/anaconda3/envs/rapidfitting/lib/python3.9/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')
/home/ferroelectric/anaconda3/envs/rapidfitting/lib/python3.9/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')


Rank 0 - 3% complete. Time remaining: 2.76 mins
Rank 0 - 6% complete. Time remaining: 2.69 mins
Rank 0 - 10% complete. Time remaining: 2.53 mins
Rank 0 - 13% complete. Time remaining: 2.39 mins
Rank 0 - 16% complete. Time remaining: 2.3 mins
Rank 0 - 20% complete. Time remaining: 2.18 mins
Rank 0 - 23% complete. Time remaining: 2.05 mins
Rank 0 - 26% complete. Time remaining: 1.95 mins
Rank 0 - 30% complete. Time remaining: 1.95 mins
Rank 0 - 33% complete. Time remaining: 1.9 mins
Rank 0 - 36% complete. Time remaining: 1.79 mins
Rank 0 - 40% complete. Time remaining: 1.74 mins
Rank 0 - 43% complete. Time remaining: 1.64 mins
Rank 0 - 46% complete. Time remaining: 1.48 mins
Rank 0 - 50% complete. Time remaining: 1.39 mins
Rank 0 - 53% complete. Time remaining: 1.33 mins
Rank 0 - 56% complete. Time remaining: 1.21 mins
Rank 0 - 60% complete. Time remaining: 1.15 mins
Rank 0 - 63% complete. Time remaining: 1.05 mins
Rank 0 - 66% complete. Time remaining: 55.68 sec
Rank 0 - 70% complete. T

...

Rank 0 - 3% complete. Time remaining: 2.59 mins
Rank 0 - 6% complete. Time remaining: 2.57 mins
Rank 0 - 10% complete. Time remaining: 2.59 mins
Rank 0 - 13% complete. Time remaining: 2.42 mins
Rank 0 - 16% complete. Time remaining: 2.31 mins
Rank 0 - 20% complete. Time remaining: 2.3 mins
Rank 0 - 23% complete. Time remaining: 2.16 mins
Rank 0 - 26% complete. Time remaining: 2.0 mins
Rank 0 - 30% complete. Time remaining: 1.93 mins
Rank 0 - 33% complete. Time remaining: 1.87 mins
Rank 0 - 36% complete. Time remaining: 1.7 mins
Rank 0 - 40% complete. Time remaining: 1.63 mins
Rank 0 - 43% complete. Time remaining: 1.53 mins
Rank 0 - 46% complete. Time remaining: 1.44 mins
Rank 0 - 50% complete. Time remaining: 1.38 mins
Rank 0 - 53% complete. Time remaining: 1.29 mins
Rank 0 - 56% complete. Time remaining: 1.24 mins
Rank 0 - 60% complete. Time remaining: 1.18 mins
Rank 0 - 63% complete. Time remaining: 1.08 mins
Rank 0 - 66% complete. Time remaining: 56.34 sec
Rank 0 - 70% complete. Ti

/home/ferroelectric/anaconda3/envs/rapidfitting/lib/python3.9/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')
/home/ferroelectric/anaconda3/envs/rapidfitting/lib/python3.9/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')


Rank 0 - 3% complete. Time remaining: 2.61 mins
Rank 0 - 6% complete. Time remaining: 2.46 mins
Rank 0 - 10% complete. Time remaining: 2.46 mins
Rank 0 - 13% complete. Time remaining: 2.32 mins
Rank 0 - 16% complete. Time remaining: 2.24 mins
Rank 0 - 20% complete. Time remaining: 2.22 mins
Rank 0 - 23% complete. Time remaining: 2.14 mins
Rank 0 - 26% complete. Time remaining: 2.01 mins
Rank 0 - 30% complete. Time remaining: 1.93 mins
Rank 0 - 33% complete. Time remaining: 1.87 mins
Rank 0 - 36% complete. Time remaining: 1.7 mins
Rank 0 - 40% complete. Time remaining: 1.61 mins
Rank 0 - 43% complete. Time remaining: 1.51 mins
Rank 0 - 46% complete. Time remaining: 1.44 mins
Rank 0 - 50% complete. Time remaining: 1.32 mins
Rank 0 - 53% complete. Time remaining: 1.22 mins
Rank 0 - 56% complete. Time remaining: 1.18 mins
Rank 0 - 60% complete. Time remaining: 1.13 mins
Rank 0 - 63% complete. Time remaining: 1.04 mins
Rank 0 - 66% complete. Time remaining: 59.04 sec
Rank 0 - 70% complete. 

/home/ferroelectric/anaconda3/envs/rapidfitting/lib/python3.9/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')
/home/ferroelectric/anaconda3/envs/rapidfitting/lib/python3.9/site-packages/BGlib/be/analysis/fitter.py:140: UserWarning: status dataset not created yet
  warn('status dataset not created yet')


Rank 0 - 6% complete. Time remaining: 2.85 mins
Rank 0 - 13% complete. Time remaining: 2.39 mins
Rank 0 - 20% complete. Time remaining: 2.28 mins
Rank 0 - 26% complete. Time remaining: 2.13 mins
Rank 0 - 33% complete. Time remaining: 1.9 mins
Rank 0 - 40% complete. Time remaining: 1.7 mins
Rank 0 - 46% complete. Time remaining: 1.59 mins
Rank 0 - 53% complete. Time remaining: 1.4 mins
Rank 0 - 60% complete. Time remaining: 1.15 mins
Rank 0 - 66% complete. Time remaining: 57.45 sec
Rank 0 - 73% complete. Time remaining: 44.59 sec
Rank 0 - 80% complete. Time remaining: 31.59 sec
Rank 0 - 87% complete. Time remaining: 19.81 sec
Rank 0 - 93% complete. Time remaining: 9.66 sec
Rank 0 - 100% complete. Time remaining: 0.0 msec
Finished processing the entire dataset!
LSQF method took 319.69779682159424 seconds to compute parameters


### Checks the results to make sure it was saved correctly

In [7]:
# print the contents of the file
dataset.print_be_tree()

/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Noisy_Data_1
    ├ Noisy_Data_2
    ├ Noisy_Data_3
    ├ Noisy_Data_4
    ├ Noisy_Data_5
    ├ Noisy_Data_6
    ├ Noisy_Data_7
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS
    ├ UDVS_Indices
├ Noisy_Data_1_SHO_Fit
  --------------------
  ├ No